In [8]:
import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.optimizers import SGD
from keras.utils import to_categorical

import numpy as np

import tensorflow as tf

import keras_tuner as kt

In [9]:
#!/usr/bin/env python3
# SPDX-License-Identifier: BSD-2-Clause
# Copyright (c) 2018 Jakub Červený
# https://github.com/jakubcerveny/gilbert/tree/master


def gilbert2d(width, height):
    """
    Generalized Hilbert ('gilbert') space-filling curve for arbitrary-sized
    2D rectangular grids. Generates discrete 2D coordinates to fill a rectangle
    of size (width x height).
    """

    if width >= height:
        yield from generate2d(0, 0, width, 0, 0, height)
    else:
        yield from generate2d(0, 0, 0, height, width, 0)


def sgn(x):
    return -1 if x < 0 else (1 if x > 0 else 0)


def generate2d(x, y, ax, ay, bx, by):

    w = abs(ax + ay)
    h = abs(bx + by)

    (dax, day) = (sgn(ax), sgn(ay)) # unit major direction
    (dbx, dby) = (sgn(bx), sgn(by)) # unit orthogonal direction

    if h == 1:
        # trivial row fill
        for i in range(0, w):
            yield(x, y)
            (x, y) = (x + dax, y + day)
        return

    if w == 1:
        # trivial column fill
        for i in range(0, h):
            yield(x, y)
            (x, y) = (x + dbx, y + dby)
        return

    (ax2, ay2) = (ax//2, ay//2)
    (bx2, by2) = (bx//2, by//2)

    w2 = abs(ax2 + ay2)
    h2 = abs(bx2 + by2)

    if 2*w > 3*h:
        if (w2 % 2) and (w > 2):
            # prefer even steps
            (ax2, ay2) = (ax2 + dax, ay2 + day)

        # long case: split in two parts only
        yield from generate2d(x, y, ax2, ay2, bx, by)
        yield from generate2d(x+ax2, y+ay2, ax-ax2, ay-ay2, bx, by)

    else:
        if (h2 % 2) and (h > 2):
            # prefer even steps
            (bx2, by2) = (bx2 + dbx, by2 + dby)

        # standard case: one step up, one long horizontal, one step down
        yield from generate2d(x, y, bx2, by2, ax2, ay2)
        yield from generate2d(x+bx2, y+by2, ax, ay, bx-bx2, by-by2)
        yield from generate2d(x+(ax-dax)+(bx2-dbx), y+(ay-day)+(by2-dby),
                              -bx2, -by2, -(ax-ax2), -(ay-ay2))


In [10]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Preprocess the data
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32') / 255

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

gilbert_curve = list(gilbert2d(28, 28))

x_train_gilbert = np.zeros((x_train.shape[0], 28, 28, 1))
x_test_gilbert = np.zeros((x_test.shape[0], 28, 28, 1))

for i in range(x_train.shape[0]):
    #index through the images starting from bottom left
    a,b = 0,0
    #loop through the gilbert curve
    for j, (x, y) in enumerate(gilbert_curve):
        #set the pixel value of the train_images to the pixel value of the train_images_orig
        x_train_gilbert[i][x][y] = x_train[i][a][b]
        #increment the index of the train_images_orig
        a += 1
        if a == 28:
            b += 1
            a = 0

for i in range(x_test.shape[0]):
    #index through the images starting from bottom left
    a,b = 0,0
    #loop through the gilbert curve
    for j, (x, y) in enumerate(gilbert_curve):
        #set the pixel value of the train_images to the pixel value of the train_images_orig
        x_test_gilbert[i][x][y] = x_test[i][a][b]
        #increment the index of the train_images_orig
        a += 1
        if a == 28:
            b += 1
            a = 0


In [11]:
def build_model(hp):

    i1 = keras.Input(shape=(28, 28, 1))
    i2 = keras.Input(shape=(28, 28, 1))

    include_orig_dropout = hp.Boolean('include_orig_dropout')
    include_gilber_dropout = hp.Boolean('include_gilber_dropout')
    orig_dropout_rate = hp.Float('orig_dropout_rate', 0.1, 0.3, step=0.05)
    gilber_dropout_rate = hp.Float('gilber_dropout_rate', 0.1, 0.3, step=0.05)

    conv_padding = hp.Choice('conv_padding', values=['same', 'valid'])

    conv1_kernels = hp.Int('conv1_kernels', 32, 256, step=32)
    conv1_kernel_size = hp.Int('conv1_kernel_size', 3, 5)
    include_conv2 = hp.Boolean('include_conv2')
    conv2_kernels = hp.Int('conv2_kernels', 32, 256, step=32)
    conv2_kernel_size = hp.Int('conv2_kernel_size', 3, 5)

    dense1_units = hp.Int('dense1_units', 32, 256, step=32)
    dense2_units = hp.Int('dense2_units', 32, 256, step=32)

    optimizer = hp.Choice('optimizer', values=['adam', 'sgd'])
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])


    if include_orig_dropout:
        x1 = keras.layers.Dropout(orig_dropout_rate)(i1)

    x1 = Conv2D(conv1_kernels, kernel_size=(conv1_kernel_size, conv1_kernel_size), padding=conv_padding, activation='relu')(x1)
    x1 = MaxPooling2D(pool_size=(2, 2))(x1)

    if include_conv2:
        x1 = Conv2D(conv2_kernels, kernel_size=(conv2_kernel_size, conv2_kernel_size), padding=conv_padding, activation='relu')(x1)

    x1 = Flatten()(x1)

    if include_gilber_dropout:
        x2 = keras.layers.Dropout(gilber_dropout_rate)(x2)

    x2 = Conv2D(conv1_kernels, kernel_size=(conv1_kernel_size, conv1_kernel_size), padding=conv_padding, activation='relu')(x2)
    x2 = MaxPooling2D(pool_size=(2, 2))(x2)

    if include_conv2:
        x2 = Conv2D(conv2_kernels, kernel_size=(conv2_kernel_size, conv2_kernel_size), padding=conv_padding, activation='relu')(x2)

    x2 = Flatten()(x2)

    x = keras.layers.concatenate([x1, x2])
    x = Dense(dense1_units, activation='sigmoid')(x)
    x = Dense(dense2_units, activation='sigmoid')(x)
    output = Dense(10, activation='softmax')(x)

    model = keras.Model(inputs=[x1, x2], outputs=output)
    
    if optimizer == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)

    model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    model.summary()
    
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=100,
    executions_per_trial=2,
    directory='gilbert',
    project_name='fashion_mnist'
)

tuner.search([x_train, x_train_gilbert], y_train, epochs=25, validation_data=([x_test, x_test_gilbert], y_test))



Reloading Tuner from gilbert/fashion_mnist/tuner0.json

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
False             |False             |include_orig_dropout
True              |True              |include_gilber_dropout
0.15              |0.2               |orig_dropout_rate
0.25              |0.2               |gilber_dropout_rate
valid             |valid             |conv_padding
192               |192               |conv1_kernels
5                 |3                 |conv1_kernel_size
True              |True              |include_conv2
160               |224               |conv2_kernels
5                 |3                 |conv2_kernel_size
256               |224               |dense1_units
224               |256               |dense2_units
adam              |adam              |optimizer
0.0001            |0.01              |learning_rate



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ keras_tensor_5CLONE │ (None, 10240)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ keras_tensor_10CLO… │ (None, 10240)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 20480)     │          0 │ keras_tensor_5CL… │
│ (Concatenate)       │                   │            │ keras_tensor_10C… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │  5,243,136 │ concatenate[1][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 224)       │     57,568 │ dense[1][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 10)        │      2,250 │ dense_1[1][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,302,954 (20.23 MB)

 Trainable params: 5,302,954 (20.23 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25


Traceback (most recent call last):
  File "/Users/thomasjones/.pyenv/versions/3.10.15/envs/MLExp/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/Users/thomasjones/.pyenv/versions/3.10.15/envs/MLExp/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/Users/thomasjones/.pyenv/versions/3.10.15/envs/MLExp/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/Users/thomasjones/.pyenv/versions/3.10.15/envs/MLExp/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/Users/thomasjones/.pyenv/versions/3.10.15/envs/MLExp/lib

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/Users/thomasjones/.pyenv/versions/3.10.15/envs/MLExp/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/Users/thomasjones/.pyenv/versions/3.10.15/envs/MLExp/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/Users/thomasjones/.pyenv/versions/3.10.15/envs/MLExp/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/Users/thomasjones/.pyenv/versions/3.10.15/envs/MLExp/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwa